In [1]:
!pip install transformers
!pip install datasets
!pip install SentencePiece
!pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 107.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 6.6 MB/s eta 0:00:00


In [1]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset

In [2]:
# Load the dataset
dataset = load_dataset('cnn_dailymail', '3.0.0', split='train[:100]')  # Load a subset of the "cnn_dailymail" dataset
# train_dataset = dataset['train']

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


In [3]:
# Load the mT5 tokenizer
tokenizer = MT5Tokenizer.from_pretrained('google/mt5-small')

You are using the legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


In [4]:
# Preprocessing function
def preprocess_function(examples):
    inputs = examples['article']
    targets = examples['highlights']
    inputs = [input_text for input_text in inputs]
    targets = [target_text for target_text in targets]
    inputs = tokenizer(inputs, padding="max_length", truncation=True, max_length=512)
    targets = tokenizer(targets, padding="max_length", truncation=True, max_length=128)
    inputs['labels'] = targets['input_ids']
    return inputs

In [5]:
train_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [6]:
train_dataset

Dataset({
    features: ['article', 'highlights', 'id', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

In [7]:
model = MT5ForConditionalGeneration.from_pretrained('google/mt5-small')

## TrainingArgument 지정
## Traininer 지정

In [8]:
# Define the training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./output_dir',
    num_train_epochs=3,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=8,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    save_total_limit=3,
)

In [9]:
# Initialize the trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=train_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8),
    ## 기본역할 -> input과 label을 동적으로 배치
    ## 입력이 최대길이보다 작을때 padding 추가하는 것
    tokenizer=tokenizer,
)

In [10]:
# Start training
trainer.train()
## 훈련손실이 없는이유는 training argument에서 지정하지 않앙ㅆ기대문
## 이후 loss가 나온다.

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
0,No log,35.617680
1,No log,33.408897
2,No log,32.614281


TrainOutput(global_step=18, training_loss=37.37480672200521, metrics={'train_runtime': 113.2445, 'train_samples_per_second': 2.649, 'train_steps_per_second': 0.159, 'total_flos': 152279910973440.0, 'train_loss': 37.37480672200521, 'epoch': 2.88})

In [11]:
model.save_pretrained('./exported_model')

In [12]:
tokenizer.save_pretrained('./exported_tokenizer')

('./exported_tokenizer/tokenizer_config.json',
 './exported_tokenizer/special_tokens_map.json',
 './exported_tokenizer/spiece.model',
 './exported_tokenizer/added_tokens.json')

In [13]:
 # Load the trained model
model = MT5ForConditionalGeneration.from_pretrained('/content/exported_model')

# Load the tokenizer
tokenizer = MT5Tokenizer.from_pretrained('/content/exported_tokenizer')

In [29]:
input_text = ["Al-Walid I (c. 674 – 715) was the sixth Umayyad caliph, ruling from October 705 until his death. The eldest son of Caliph Abd al-Malik (r. 685–705), he continued his father's efforts to centralize government, impose a more Arabic and Islamic character on the state, and expand its borders. He heavily depended on al-Hajjaj ibn Yusuf, his father's powerful viceroy over the eastern half of the caliphate. During his reign, armies commissioned by al-Hajjaj conquered Sind and Transoxiana in the east, while the troops of Musa ibn Nusayr, the governor of Ifriqiya, conquered the Maghreb and Hispania in the west, bringing the caliphate to its largest territorial extent. Al-Walid financed impressive public works, including the Great Mosque of Damascus, the al-Aqsa Mosque in Jerusalem, and the Prophet's Mosque in Medina. He was the first caliph to institute programs for social welfare, and his reign was marked by domestic peace and prosperity. (Full article...)"]
input_encoding = tokenizer(input_text, truncation=True, padding="max_length", max_length=2048, return_tensors="pt")

# Generate prediction
output = model.generate(input_encoding["input_ids"])
output


tensor([[     0, 250099,    259,  33295,      1]])

In [30]:
predicted_text = tokenizer.batch_decode(output, skip_special_tokens=True)
predicted_text

['<extra_id_0> 705']

In [42]:
model = MT5ForConditionalGeneration.from_pretrained('google/mt5-base')
tokenizer = MT5Tokenizer.from_pretrained('google/mt5-base')
input_text = """[방콕=연합뉴스) 강종훈 특파원 = 5월 태국 총선에서 돌풍을 일으키며 제1당에 오른 전진당(MFP) 피타 림짜른랏 대표의 총리 도전이 멈추게 됐다.

  전진당을 비롯한 야권 8개 연합은 19일 상·하원 총리 선출 2차 투표에 지난 13일 과반 획득에 실패한 피타 대표를 후보로 재지명했으나, 투표 자체가 무산됐다.

  타이PBS 등 현지 매체와 외신에 따르면 군부 진영 의원들은 한차례 거부된 안을 다시 제출할 수 없다며 후보 재지명 불가를 주장했다.

  야권은 총리 후보 지명은 법안을 다시 발의하는 것과 다르다고 반박했다.

  8시간 넘는 찬반 토론 끝에 의회는 피타 후보의 재지명 가능 여부를 묻는 표결을 시작했다.

  총리 선출 1차 투표와 마찬가지로 피타 후보에 반대하는 의원들의 표가 우세했다.

  전진당 측 의원들의 반발 속에 완 노르 마타 하원의장은 "이번 회기에 피타 대표가 두 번 총리 후보로 지명될 수 없다"고 밝혔다.

  게다가 피타 대표는 토론 도중 헌법재판소의 그에 대한 의원 직무 정지 결정으로 의회를 떠나야 했다.

  앞서 선거관리위원회가 미디어 기업 주식을 보유한 피타 대표의 총선 출마가 위법이라며 헌재에 사건을 회부했고, 헌재는 이날 이를 받아들이고 판결 때까지 그의 의원 직무를 정지시켰다.

  헌재 판결에 따라 피타 대표는 의원직을 박탈당할 수 있다.

  헌재는 전진당의 왕실모독죄 개정 추진에 대한 위헌 여부도 심리할 예정이어서 최악의 경우 정당 해산 가능성도 있다.

  피타 대표는 동료 의원들과 인사하고 주먹을 들어 보이며 의회를 빠져나갔다.

  그는 자신이 의회에서 퇴장하는 장면을 찍은 사진을 소셜미디어(SNS)에 올리며 "돌아오겠다(I'll be back)"라고 썼다"""
input_encoding = tokenizer(input_text, padding="max_length", max_length=512, return_tensors="pt")

# Generate prediction
output = model.generate(input_encoding["input_ids"])
output

tensor([[     0, 250099,    259,    649, 134141,    259,  32880,   2924,   1000,
              1]])

In [41]:
predicted_text = tokenizer.batch_decode(output, skip_special_tokens=True)
predicted_text

['<extra_id_0> 의회가 끝나고']